In [125]:
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
import kornia.augmentation as K
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from projects.VolcanoFinder.web.models import MyFirstCNN
import matplotlib.pyplot as plt

In [126]:
"""
To learn:
- Look at feature maps (patterns)
"""

cpu_transforms = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root='C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/train', transform=cpu_transforms)
val_dataset = ImageFolder(root='C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/val', transform=cpu_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=12,
    prefetch_factor=6,
    persistent_workers=True,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=12,
    prefetch_factor=6,
    persistent_workers=True,
    pin_memory=True
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gpu_augmentations = nn.Sequential(
    K.Resize((128, 128)),
    K.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    K.Normalize(mean=torch.tensor([0.5, 0.5, 0.5], device=device),
                std=torch.tensor([0.5, 0.5, 0.5], device=device))
).to(device)

# ------------------------------- #
# Tried, but disadvantageous: RandRotation, RandGaussianNoise (Terrible, barely above 50% accuracy), ColorJitter, RandGauBlur, RandSolarize (- ~6%)
# ------------------------------- #

In [127]:
# ------------------------------- #
best_accuracy: float = 0.0
results: list = []
runs: int = 200

for i in range(runs):
    print(f"===================== RUN: {i+1}/{runs} =====================")
    model = MyFirstCNN().to(device)
    criterion = nn.BCELoss()

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    # ------------TRAIN--------------- #
    num_epochs: int = 20
    best_val_loss: float = math.inf

    for epoch in range(num_epochs):
        # Training loop
        model.train()
        running_loss: float = 0.0
        for images, labels in train_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            images = gpu_augmentations(images)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        #print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader):.4f}")

        # ------------VALIDATE--------------- #
        model.eval()
        val_correct: int = 0
        val_total: int = 0
        val_loss: float = 0.0
        with torch.inference_mode():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

                outputs = model(images)
                val_loss += criterion(outputs, labels.float()).item()

                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss: float = val_loss / len(val_loader)
        val_accuracy: float = 100 * val_correct / val_total

        current_best_model_path = f'volcano_finder_myowncnn_run_{i+1}_best.pth'

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), current_best_model_path)
            #print(f" ------------- Best model saved | Epoch: {epoch + 1}, Val loss: {avg_val_loss:.4f} ------------- ")

        #print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

        scheduler.step(avg_val_loss)

    # ------------TEST--------------- #

    test_data_path = 'C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/test_images'
    test_dataset = ImageFolder(root=test_data_path, transform=cpu_transforms)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    loaded_model = MyFirstCNN().to(device)

    loaded_model.load_state_dict(torch.load(current_best_model_path))

    loaded_model.to(device)
    loaded_model.eval()


    test_correct: int = 0
    test_total: int = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = loaded_model(images)
            predicted = (outputs > 0.5).float()
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    accuracy: float = 100 * test_correct / test_total
    results.append(accuracy)
    print('\n# ------------------------------- #\n')
    print(f"Test Accuracy: {accuracy:.2f}%\n")

    if (accuracy > best_accuracy):
        best_accuracy = accuracy
        print(f"\nNew best: {best_accuracy:.2f}%\n\n")
        torch.save(loaded_model.state_dict(), "best_volcano_finder.pth")

    torch.cuda.empty_cache()
    if os.path.exists(current_best_model_path):
        os.remove(current_best_model_path)

plt.plot(results)
plt.xlabel('Runs')
plt.ylabel('Accuracy')
plt.show()
print('Done!')
print(f'\n\nBest accuracy: {best_accuracy:.2f}%\n')
print(sum(results)/len(results))
# ------------------------------- #

===================== RUN: 1/200 =====================

# ------------------------------- #

Test Accuracy: 62.84%


New best: 62.84%


===================== RUN: 2/200 =====================

# ------------------------------- #

Test Accuracy: 76.23%


New best: 76.23%


===================== RUN: 3/200 =====================

# ------------------------------- #

Test Accuracy: 77.05%


New best: 77.05%


===================== RUN: 4/200 =====================

# ------------------------------- #

Test Accuracy: 77.60%


New best: 77.60%


===================== RUN: 5/200 =====================

# ------------------------------- #

Test Accuracy: 75.41%

===================== RUN: 6/200 =====================

# ------------------------------- #

Test Accuracy: 66.94%

===================== RUN: 7/200 =====================

# ------------------------------- #

Test Accuracy: 71.58%

===================== RUN: 8/200 =====================

# ------------------------------- #

Test Accuracy: 

KeyboardInterrupt: 